## Set up environment variables
Set PROJECT_ID to your project.

In [1]:
import sys
import os
PROJECT_ID = "genie-ui-dev"
os.environ["PROJECT_ID"] = PROJECT_ID

## Clone GENIE repository

In [2]:
#!git clone https://github.com/GoogleCloudPlatform/core-solution-services
#!cd core-solution-services/components/llm_service/notebooks

## Import GENIE code

In [3]:
sys.path.append("../../common/src")
sys.path.append("../src")
from common.models import QueryEngine
from services.query.web_datasource import WebDataSource
from services.query.vertex_search import build_vertex_search

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_

## Set web source url and create GENIE query engine
Set `depth_limit` to the depth which you want to crawler to follow links in the site.  `depth_limit = 0` means only crawl the page pointed to in the URL, `depth_limit = 1` means crawl each link present in the page pointed to by the URL, `depth_limit = 2` means crawl each link in the pages linked to from the first page, etc.

In [4]:
data_url = "https://dmv.nv.gov/"
depth_limit = 1
q_engine = QueryEngine(name="test web download", doc_url=data_url, params={"depth_limit":depth_limit})

## Download files to bucket

In [6]:
import tempfile
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)
bucket_name = WebDataSource.downloads_bucket_name(q_engine)
web_datasource = WebDataSource(storage_client, bucket_name=bucket_name, depth_limit=depth_limit)
with tempfile.TemporaryDirectory() as temp_dir:
    data_source_files = web_datasource.download_documents(data_url, temp_dir)

print(f"Downloaded {len(data_source_files)} files to gs://{bucket_name}")

INFO: [utils/gcs_helper.py:54 - create_bucket()] Bucket genie-ui-dev-downloads-test-web-download already exists.
INFO: [utils/gcs_helper.py:30 - clear_bucket()] Deleting all objects from GCS bucket genie-ui-dev-downloads-test-web-download
INFO: [utils/gcs_helper.py:37 - clear_bucket()] 39 files deleted
INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_

2024-04-26 08:15:51 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-04-26 08:15:51 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.12.6, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.13 (v3.9.13:6de2ca5339, May 17 2022, 11:37:23) - [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform macOS-14.4.1-arm64-arm-64bit
2024-04-26 08:15:51 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-26 08:15:51 [py.warnings] WARNING: /Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compat

INFO: [query/web_datasource.py:49 - save_content()] Saving .html to /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpn3wiedbw
INFO: [query/web_datasource.py:53 - save_content()] 13353 bytes written
INFO: [utils/gcs_helper.py:76 - upload_to_gcs()] Uploading /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpn3wiedbw/.html to GCS bucket genie-ui-dev-downloads-test-web-download


...

INFO: [utils/gcs_helper.py:82 - upload_to_gcs()] Uploaded /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpn3wiedbw/dlduplicate.html to gs://genie-ui-dev-downloads-test-web-download/dlduplicate.html
INFO: [query/web_datasource.py:224 - _item_scraped()] Downloaded Response URL: https://dmv.nv.gov/dlduplicate.htm


2024-04-26 08:16:03 [utils/gcs_helper.py] INFO: Uploaded /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpn3wiedbw/dlduplicate.html to gs://genie-ui-dev-downloads-test-web-download/dlduplicate.html
2024-04-26 08:16:03 [query/web_datasource.py] INFO: Downloaded Response URL: https://dmv.nv.gov/dlduplicate.htm
2024-04-26 08:16:03 [scrapy.core.engine] INFO: Closing spider (finished)
2024-04-26 08:16:03 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 10104,
 'downloader/request_count': 40,
 'downloader/request_method_count/GET': 40,
 'downloader/response_bytes': 871443,
 'downloader/response_count': 40,
 'downloader/response_status_count/200': 40,
 'dupefilter/filtered': 4,
 'elapsed_time_seconds': 11.883224,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2024, 4, 26, 12, 16, 3, 293522, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 122068,
 'httpcompression/response_count': 4,
 'item_scraped_count': 39,
 'log_count/I

finished
https://dmv.nv.gov/
https://dmv.nv.gov/regoutofstate.htm
https://dmv.nv.gov/regdealer.htm
https://dmv.nv.gov/nvreg.htm
https://dmv.nv.gov/nvregreq.htm
https://dmv.nv.gov/regprivate.htm
https://dmv.nv.gov/platesveterans.htm
https://dmv.nv.gov/platesdisabled.htm
https://dmv.nv.gov/former.htm
https://dmv.nv.gov/mchome.htm
https://dmv.nv.gov/motorcycle.htm
https://dmv.nv.gov/miloosres.htm
https://dmv.nv.gov/newresident.htm
https://dmv.nv.gov/dltrafficlaws.htm
https://dmv.nv.gov/codebook.htm
https://dmv.nv.gov/dlschools.htm
https://dmv.nv.gov/dltesting.htm
https://dmv.nv.gov/dlhandbook.htm
https://dmv.nv.gov/nvdl.htm
https://dmv.nv.gov/assessor.htm
https://dmv.nv.gov/locat.htm
https://dmv.nv.gov/cdl.htm
https://dmv.nv.gov/dlhistory.htm
https://dmv.nv.gov/realid.htm
https://dmv.nv.gov/canceldrive.htm
https://dmv.nv.gov/cdloffices.htm
https://dmv.nv.gov/dmvforms.htm
https://dmv.nv.gov/platesmain.htm
https://dmv.nv.gov/floodvehicles.htm
https://dmv.nv.gov/salvage.htm
https://dmv.nv.go

Program shutting down, attempting to send 353 queued log entries to Cloud Logging...
Waiting up to 5 seconds.
Failed to send 34 pending logs.


INFO: [query/web_datasource.py:276 - download_documents()] Scraped 39 links
Downloaded 39 files to gs://genie-ui-dev-downloads-test-web-download


## Build Vertex Agent Builder app and data source

Make sure you have enabled Vertex Agent Builder in your project:

https://console.cloud.google.com/gen-app-builder?project=$PROJECT_ID

In [7]:
build_vertex_search(q_engine)

INFO: [query/vertex_search.py:196 - build_vertex_search()] Building vertex search engine [test web download] [https://dmv.nv.gov/]
INFO: [query/vertex_search.py:521 - download_web_docs()] downloading web docs to bucket [genie-ui-dev-downloads-test-web-download]
INFO: [utils/gcs_helper.py:54 - create_bucket()] Bucket genie-ui-dev-downloads-test-web-download already exists.
INFO: [utils/gcs_helper.py:30 - clear_bucket()] Deleting all objects from GCS bucket genie-ui-dev-downloads-test-web-download
INFO: [utils/gcs_helper.py:37 - clear_bucket()] 39 files deleted
INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_

2024-04-26 08:16:37 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-04-26 08:16:38 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.12.6, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.13 (v3.9.13:6de2ca5339, May 17 2022, 11:37:23) - [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform macOS-14.4.1-arm64-arm-64bit
2024-04-26 08:16:38 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-26 08:16:38 [py.warnings] WARNING: /Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compat

INFO: [query/web_datasource.py:49 - save_content()] Saving .html to /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpvjh8hhbw
INFO: [query/web_datasource.py:53 - save_content()] 13353 bytes written
INFO: [utils/gcs_helper.py:76 - upload_to_gcs()] Uploading /var/folders/7s/s_wkt16s1_7cdm2p_ptskdl400dhhl/T/tmpvjh8hhbw/.html to GCS bucket genie-ui-dev-downloads-test-web-download


...

Program shutting down, attempting to send 283 queued log entries to Cloud Logging...
Waiting up to 5 seconds.
Sent all pending logs.


INFO: [query/web_datasource.py:276 - download_documents()] Scraped 39 links
INFO: [query/vertex_search.py:374 - import_documents_to_datastore()] Waiting for import operation to complete: projects/779118494750/locations/global/collections/default_collection/dataStores/test-web-download/branches/0/operations/import-documents-7749253967306633950
INFO: [query/vertex_search.py:378 - import_documents_to_datastore()] document import result: <bound method PollingFuture.result of <google.api_core.operation.Operation object at 0x171174c70>>
INFO: [query/vertex_search.py:384 - import_documents_to_datastore()] document metadata from import: create_time {
  seconds: 1714133820
  nanos: 537760000
}
update_time {
  seconds: 1714134424
  nanos: 393996000
}
success_count: 39
total_count: 39

INFO: [query/vertex_search.py:240 - build_vertex_search()] Created vertex search engine for test web download
ERROR: [query/vertex_search.py:258 - build_vertex_search()] Error building vertex search query engine [I

Traceback (most recent call last):
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/services/query/vertex_search.py", line 244, in build_vertex_search
    q_engine.update()
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../../common/src/common/models/base_model.py", line 90, in update
    return super().update(key, transaction, batch)
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/fireo/models/model.py", line 407, in update
    raise InvalidKey(
fireo.queries.errors.InvalidKey: Invalid key to update model "QueryEngine" 


ERROR: [query/vertex_search.py:466 - delete_vertex_search()] Exception deleting datastore test-web-download: 400 Request contains an invalid argument.
INFO: [query/vertex_search.py:470 - delete_vertex_search()] deleting datastore test-web-download
ERROR: [query/vertex_search.py:477 - delete_vertex_search()] Exception deleting datastore test-web-download: 400 Request contains an invalid argument.


InvalidKey: Invalid key to update model "QueryEngine" 